In [3]:
import os
import pathlib
from csv import DictReader

import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import *

In [4]:
def build_network(width, height, depth, classes):
    input_layer = Input(shape=(width, height, depth))

    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.25)(x)

    x = Dense(units=classes)(x)
    output = Activation('sigmoid')(x)

    return Model(input_layer, output)

In [46]:
def load_images_and_labels(image_paths, styles, target_size):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=target_size)
        image = img_to_array(image)
        image_id = image_path.split(os.path.sep)[-1][:-4]

        image_style = styles[image_id]
#         label = (image_style['gender'], image_style['usage']) 
        #adding more labels
        label = (image_style['gender'], image_style['usage'], image_style['articleType'],image_style['season'],image_style['year']) 

        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

In [47]:
SEED = 999
np.random.seed(SEED)

base_path = '../input/fashion-product-images-small'
styles_path = str(base_path +'/'+'styles.csv')
images_path_pattern = str(base_path+'/'+'images/*.jpg')
image_paths = glob.glob(images_path_pattern)

In [48]:
with open(styles_path, 'r') as f:
    dict_reader = DictReader(f)
    STYLES = [*dict_reader] #list of ordered dicts as shown below
    """ 
    OrderedDict([('id', '15970'), ('gender', 'Men'), ('masterCategory', 'Apparel'), ('subCategory', 'Topwear'), 
    ('articleType', 'Shirts'), ('baseColour', 'Navy Blue'), ('season', 'Fall'), ('year', '2011'), ('usage', 'Casual'),
    ('productDisplayName', 'Turtle Check Men Navy Blue Shirt')])
    """
    article_type = {'Watches'} # ,'Casual Shoes''Shirts','Tshirts','Sports Shoes'
    genders = {'Men', 'Women','Unisex','Boys','Girls'}
    usages = {'Casual', 'Smart Casual', 'Formal'}
    
    STYLES = {style['id']: style
              for style in STYLES
              if (style['articleType'] in article_type and
                  style['gender'] in genders and
                  style['usage'] in usages)}

In [49]:
for key,val in STYLES.items():
    print(f"key:> {key} value:> {val}")
    break
print("The size of STYLES:> ",len(STYLES))

In [50]:
image_paths = [*filter(lambda p: p.split(os.path.sep)[-1][:-4] in STYLES.keys(),image_paths)]
print("the length of image_path list:> ",len(image_paths))

In [51]:
X, y = load_images_and_labels(image_paths, STYLES, (64, 64))
X = X.astype('float') / 255.0

In [52]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

In [53]:
print(mlb.classes_,len(mlb.classes_))


In [55]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y,
                                     test_size=0.2,
                                     random_state=SEED)
(X_train, X_valid,
 y_train, y_valid) = train_test_split(X_train, y_train,
                                      test_size=0.2,
                                      random_state=SEED)

In [56]:
model = build_network(width=64,
                      height=64,
                      depth=3,
                      classes=len(mlb.classes_))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [66]:
BATCH_SIZE = 64
EPOCHS = 1000
model.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=BATCH_SIZE,
          epochs=EPOCHS)

In [68]:
result = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print(f'Test accuracy: {result[1]}')

test_image = np.expand_dims(X_test[0], axis=0)
probabilities = model.predict(test_image)[0]
print(probabilities)

In [69]:
for label, p in zip(mlb.classes_, probabilities):
    if p > 0.5:
        print(f'{label}: {p * 100:.2f}%')

In [70]:
ground_truth_labels = np.expand_dims(y_test[0], axis=0)
ground_truth_labels = mlb.inverse_transform(ground_truth_labels)
print(f'Ground truth labels: {ground_truth_labels}')

# Inferencing 

In [71]:
import matplotlib.pyplot as plt
image = load_img(r'../input/fashion-product-images-small/images/45258.jpg', target_size=(64,64))
plt.imshow(image)

In [72]:
image = img_to_array(image)
image= image/255.0
image = np.expand_dims(image, axis=0)
probabilities= model.predict(image)[0]

In [73]:
probabilities

In [74]:
for label, p in zip(mlb.classes_, probabilities):
    if p > 0.5:
        print(f'{label}: {p * 100:.2f}%')
        
#Ground truth
STYLES['45258']

In [55]:
# ground_truth_labels = np.expand_dims(y_test[0], axis=0)
# ground_truth_labels = mlb.inverse_transform(ground_truth_labels)
# print(f'Ground truth labels: {ground_truth_labels}')

In [77]:
# Inferencing 2

image = load_img(r'../input/fashion-product-images-small/images/45293.jpg', target_size=(64,64))
plt.imshow(image)

image = img_to_array(image)
image= image/255.0
image = np.expand_dims(image, axis=0)
probabilities= model.predict(image)[0]

for label, p in zip(mlb.classes_, probabilities):
    if p > 0.5:
        print(f'{label}: {p * 100:.2f}%')
        
#Ground truth
print("Ground truth:> ",STYLES['45293'])

'gender','usage','articleType','subCategory'